In [ ]:
import numpy as np
import os
from SPE3reading.py import SPE3map
import matplotlib.pyplot as plt
import spectrapepper as spep
import re
import csv
from scipy.signal import find_peaks

Load the data

In [ ]:
dir_name = 'C:/Users/emmah/OneDrive - University of Cambridge/Data Jan 2026/Sample 7/Zero B field/PL capped'
file_name = 'sample7_uncapped_emitter01_grating150_exposure30_100uW 2026 January 12 19_26_02.spe'

full_path = os.path.join(dir_name, file_name)
SPE = SPE3map(fname = full_path)
wavelength = SPE.wavelength
SPE_clean = SPE.data[:,0,:] # Remove noise

for i in range (0, np.shape(SPE_clean)[0]): #plot all the frames to visualise 
    plt.figure()
    plt.plot(wavelength, SPE_clean[i])
    plt.title(f"Frame {i}")
    plt.xlabel('Wavelength (nm)')
    plt.ylabel('Intensity (counts)')

Basic preprocessing - cosmic ray removal, background subtraction, normalisation

In [ ]:
frames_to_remove = []  #if some frames were just noise (e.g. emitter died) we remove them from the dataset
SPE_clean = np.delete(SPE_clean, frames_to_remove, axis=0)

cosmic_frames = [] #list of frames with cosmic rays that need to be removed
SPE_clean2 = SPE_clean #new array containing spectra after being cleaned of cosmic rays
for i in cosmic_frames:
    spectra_list = [SPE_clean[i-1], SPE_clean[i], SPE_clean[i+1]] #y, x  # y = spectras in rows, x = axis
    y = spep.cosmicmed(spectra_list, sigma=1.7)  # elimination of cosmic rays by similarity between frames i-1, i and i+1
    plt.figure()
    plt.plot(wavelength, y[1])
    plt.title(f"Frame {i} after cosmic rays removed") #plot to check that the cosmic ray removal hasn't altered the rest of the spectrum - change sigma for stronger and less strong removal
    plt.xlabel('Wavelength (nm)')
    plt.ylabel('Intensity (counts)')
    xlim = plt.gca().get_xlim()
    ylim = plt.gca().get_ylim()

    plt.figure()
    plt.plot(wavelength, SPE_clean[i])
    plt.title(f"Frame {i} before cosmic rays removed")
    plt.xlabel('Wavelength (nm)')
    plt.ylabel('Intensity (counts)')
    ax = plt.gca()
    ax.set_xlim(xlim) #set same axes limits as previous plot to double check that the rest of the spectrum is unaffected
    ax.set_ylim(ylim)
    SPE_clean2[i] = y[1] #replace old spectrum with new spectrum without cosmic rays
    
#Spectra background subtraction and averaging
spectra = SPE_clean2
background = np.mean(spectra[:, 10:200], axis=1)  # background levels given by beginning of spectrum ~500nm, one value per frame
spectra_bg_sub = spectra - background[:, None]
av_spectra = np.mean(spectra_bg_sub, axis=0) #take average of frames to get one spectrum

plt.figure()
plt.plot(wavelength, av_spectra)
plt.title('Averaged spectrum')
plt.xlabel('Wavelength (nm)')
plt.ylabel('Intensity (counts)')

#Spectrum normalisation
spectra_norm = av_spectra / np.max(av_spectra)


Peak detection

In [ ]:
peaks, properties = find_peaks(spectra_norm, height = 0.1, prominence = 0.1)
print("Intensity of peaks detected (counts):", spectra_norm[peaks])
print("Wavelength of peaks detected (nm):", wavelength[peaks])

peak_heights = properties["peak_heights"]
sorted_indices = np.argsort(peak_heights)[::-1] #sort the indices in descending order according to the peak height
peaks_top = peaks[sorted_indices[:10]] #select the first ten peaks of interest
spectra_top = spectra_norm[peaks_top]
wavelength_top = wavelength[peaks_top]

print("Intensity of ZPL and sideband peak (counts):", spectra_top)
print("Wavelength of ZPL and sideband peak (nm):", wavelength_top)

zpl_region = (wavelength_top > 610) & (wavelength_top < 641) #finding the ZPL peak by considering +/- 5nm from 622nm
zpl_intensity = float(spectra_top[zpl_region][0])
zpl_wavelength = float(wavelength_top[zpl_region][0])
print("Intensity and wavelength of ZPL peak (counts, nm):", zpl_intensity, zpl_wavelength)

psb_region = (wavelength_top > 665) #for PSB peak around 170 meV
psb_intensity = float(spectra_top[psb_region][0])
psb_wavelength = float(wavelength_top[psb_region][0])
print("Intensity and wavelength of PSB peak (counts, nm):", psb_intensity, psb_wavelength)

relative_intensity = zpl_intensity / psb_intensity

plt.figure()
plt.plot(wavelength, spectra_norm)
plt.plot(wavelength[peaks], spectra[peaks], "x", color = 'red') #all peaks detected
plt.plot(zpl_wavelength, zpl_intensity, "x", color = 'green') #ZPL peak picked out due to highest intensity
plt.plot(psb_wavelength, psb_intensity, "x", color = 'green') #PSB peak picked out around 170 meV
parts = base.split()
plt.title(title)
plt.xlabel('Wavelength (nm)')
plt.ylabel('Intensity (counts)')
plt.show()


Creating standard plots for the spectrum

Generating classification for the emitter (1 ZPL, 2 ZPL, 3 ZPL, uncertain)

Inputting important info into csv file